In [1]:
import sys
import pathlib

import folium
import pandas as pd
import geopandas as gpd

from datetime import datetime

DIR = pathlib.Path('..')
sys.path.append(str(DIR))

import transportanalyst as ta

DATA_DIR = DIR/'data/'
OUT_DIR = DIR/'output/'

%load_ext autoreload
%autoreload 2

In [2]:
f = open(DATA_DIR/"secrets.txt", "r")
here_key = f.readline().split(': ')[1].rstrip()
google_key = f.readline().split(': ')[1].rstrip()

In [13]:
#Read some random locations to be used for routing
gdf = gpd.read_file(DATA_DIR/'cycle.geojson')
gdf['name'] = ['loc1', 'loc2', 'loc3']
gdf

map1 = folium.Map(location=[-36.850806,174.76456], zoom_start=12)
folium.GeoJson(gdf).add_to(map1)
map1

In [14]:
#Use google api to find the best route best route.
google_route = ta.google_route(
    in_gdf = gdf, 
    trip_name = 'test',
    mode = 'cycle',
    api_key=google_key,
)
folium.GeoJson(google_route).add_to(map1)
map1

In [5]:
#Use here api to find the best route best route.
ta.here_route(
    in_gdf = gdf, 
    trip_name = 'test',
    mode = 'public_transport',
    api_key=here_key,
)

{'mode': 'fastest;publicTransport', 'departure': '2020-09-08T12:08:00', 'apiKey': 'gnxn1zfVqr3j-MhK5unP8ezHcBvbbEeYeNnNt2TMC68', 'waypoint0': '-36.86671222190732,174.74544525146484', 'waypoint1': '-36.81917959744267,174.6390151977539', 'waypoint2': '-36.896645347849706,174.82131958007812'}


{'response': {'metaInfo': {'timestamp': '2020-09-08T00:08:39Z',
   'mapVersion': '8.30.112.152',
   'moduleVersion': '7.2.202035-7741',
   'interfaceVersion': '2.6.76',
   'availableMapVersion': ['8.30.112.152']},
  'route': [{'waypoint': [{'linkId': '-865017866',
      'mappedPosition': {'latitude': -36.8666151, 'longitude': 174.7453927},
      'originalPosition': {'latitude': -36.8667123, 'longitude': 174.7454453},
      'type': 'stopOver',
      'spot': 0.1818182,
      'sideOfStreet': 'left',
      'mappedRoadName': 'Home St',
      'label': 'Home St',
      'shapeIndex': 0,
      'source': 'user'},
     {'linkId': '+737064525',
      'mappedPosition': {'latitude': -36.819005, 'longitude': 174.6394551},
      'originalPosition': {'latitude': -36.8191796, 'longitude': 174.6390152},
      'type': 'stopOver',
      'spot': 0.4460432,
      'sideOfStreet': 'left',
      'mappedRoadName': 'Raphael Pl',
      'label': 'Raphael Pl',
      'shapeIndex': 308,
      'source': 'user'},
     {

In [ ]:
#otp doesn't have the waypoints now.
otp_route = ta.otp_route(
    in_gdf = gdf, 
    trip_name = 'test',
    date_time = datetime.strptime('20200109 0800', '%Y%m%d %H%M'),
    mode = 'public_transport',
)
folium.GeoJson(otp_route[['geometry']]).add_to(m)
m

In [ ]:
otp_service_area = ta.otp_service_area(
    in_gdf = gdf,
    mode = 'public_transport', 
    breaks = [10, 20],
    date_time = datetime.strptime('20200109 0800', '%Y%m%d %H%M'),
)
folium.GeoJson(otp_service_area[['geometry']]).add_to(m)
m

In [15]:
here_service_area= ta.here_service_area(
    in_gdf = gdf,
    id_field = 'name',
    mode = 'walk', 
    breaks = [10, 20],
    api_key=here_key,
)
folium.GeoJson(here_service_area[['geometry']]).add_to(map1)
map1

In [21]:
#Read census data to be used for accessibility
census = gpd.read_file(DATA_DIR/'census_sa1.geojson')

map2 = folium.Map(location=[-36.850806,174.76456], zoom_start=14)
folium.GeoJson(census).add_to(map2)
map2

In [25]:
ta.accessibility(
    census = census,
    fields = ['pow2018', 'pop2018'],
    mode = 'walk',
    travel_time = 5,
    api = 'here',
    api_key = here_key,
    show_progress = True,
)

9.9% is done.
19.9% is done.
29.8% is done.
30.5% is done.
39.7% is done.
40.4% is done.
49.7% is done.
50.3% is done.
59.6% is done.
60.3% is done.
69.5% is done.
70.2% is done.
80.1% is done.
90.1% is done.
100.0% is done.


,SA12018_V1,pow2018,pop2018,geometry,centroid,__orig_area__,walk_20200908_1232_pow2018,walk_20200908_1232_pop2018
0,7005275,18.0,78,"MULTIPOLYGON (((174.76109 -36.84738, 174.76141...",POINT (174.76131 -36.84766),4.046591e-07,10436.200534,2496.703147
1,7005339,12.0,135,"MULTIPOLYGON (((174.76261 -36.85667, 174.76304...",POINT (174.76257 -36.85693),2.658419e-07,3351.621289,3221.512746
2,7005533,120.0,108,"MULTIPOLYGON (((174.76817 -36.85409, 174.76770...",POINT (174.76695 -36.85415),9.374136e-07,3801.829599,2922.228555
3,7005513,15.0,192,"MULTIPOLYGON (((174.77149 -36.84715, 174.77119...",POINT (174.77122 -36.84705),1.869946e-07,7036.374020,3036.144101
4,7005133,135.0,372,"MULTIPOLYGON (((174.75542 -36.85729, 174.75546...",POINT (174.75520 -36.85794),8.197824e-07,1460.980482,945.372797
...,...,...,...,...,...,...,...,...
146,7005548,36.0,21,"MULTIPOLYGON (((174.76861 -36.85530, 174.76881...",POINT (174.76781 -36.85604),1.668605e-06,882.291318,1337.400327
147,7005549,27.0,477,"MULTIPOLYGON (((174.76728 -36.85672, 174.76701...",POINT (174.76693 -36.85644),5.771558e-07,1341.235885,1965.841236
148,7005689,906.0,411,"MULTIPOLYGON (((174.77925 -36.85349, 174.77967...",POINT (174.77935 -36.85587),1.091714e-05,885.038140,380.940437
149,7005654,3231.0,252,"MULTIPOLYGON (((174.76519 -36.84217, 174.76523...",POINT (174.77907 -36.84385),9.086832e-05,145.247102,11.328465


In [11]:
ta.catchment_pop(
    catchment = here_service_area, 
    census = census, 
    fields = ['pow2018'],
    show_progress = True,
)

100% is done.


,name,time,geometry,pow2018
0,loc1,600,"POLYGON ((174.70751 -36.87012, 174.70871 -36.8...",12940.405592
1,loc1,1200,"POLYGON ((174.60863 -36.80832, 174.60949 -36.8...",106899.284042
2,loc2,600,"POLYGON ((174.60417 -36.81828, 174.60468 -36.8...",NaN
3,loc2,1200,"POLYGON ((174.55370 -36.77399, 174.55456 -36.7...",NaN
4,loc3,600,"POLYGON ((174.77291 -36.86806, 174.77394 -36.8...",NaN
5,loc3,1200,"POLYGON ((174.66099 -36.85913, 174.66339 -36.8...",104077.317815
